In [50]:
#!/usr/bin/env python3
"""
Генератор данных для схемы Logistics & Supply Chain в ClickHouse
Генерирует репрезентативные данные для всех слоев: RAW, STAGE, MART
"""

import random
import json
from datetime import datetime, timedelta, date
from typing import List, Dict, Any
import uuid

try:
    from faker import Faker
    from faker.providers import internet, company, address, lorem
except ImportError:
    print("Установите faker: pip install faker")
    exit(1)

try:
    import clickhouse_connect
except ImportError:
    print("Установите clickhouse-connect: pip install clickhouse-connect")
    exit(1)

# Инициализация
fake = Faker('ru_RU')
Faker.seed(42)
random.seed(42)

# Конфигурация
CLICKHOUSE_HOST = 'localhost'
CLICKHOUSE_PORT = 8123
CLICKHOUSE_USER = 'admin'
CLICKHOUSE_PASSWORD = 'admin123'
CLICKHOUSE_DB = 'logistics'

# Параметры генерации
NUM_REGIONS = 20
NUM_WAREHOUSES = 10
NUM_SUPPLIERS = 15
NUM_PRODUCTS = 50
NUM_CUSTOMERS = 200
NUM_ORDERS = 5000  # ~5000 заказов
NUM_SHIPMENTS = 4000  # ~4000 поставок
NUM_INVENTORY_SNAPSHOTS = 10000  # ~10000 снимков инвентаря
NUM_PRODUCT_PRICES = 200  # ~200 записей цен
NUM_CUSTOMER_SEGMENTS = 150  # ~150 сегментов клиентов

# Глобальные справочники (заполняются при генерации)
regions_dict: Dict[int, Dict] = {}
warehouses_list: List[str] = []
suppliers_list: List[str] = []
products_list: List[str] = []
customers_list: List[str] = []


def init_client():
    """Инициализация клиента ClickHouse"""
    try:
        client = clickhouse_connect.get_client(
            host=CLICKHOUSE_HOST,
            port=CLICKHOUSE_PORT,
            username=CLICKHOUSE_USER,
            password=CLICKHOUSE_PASSWORD,
            database=CLICKHOUSE_DB
        )
        print(f"✓ Подключение к ClickHouse установлено: {CLICKHOUSE_HOST}:{CLICKHOUSE_PORT}")
        return client
    except Exception as e:
        print(f"✗ Ошибка подключения к ClickHouse: {e}")
        exit(1)


def generate_regions(client):
    """Генерация иерархической структуры регионов"""
    print("\n📍 Генерация регионов...")
    
    regions = []
    region_id = 1
    now = datetime.now()
    
    # Страны (корневые элементы)
    countries = ['Россия', 'Казахстан', 'Беларусь']
    country_ids = []
    
    for country in countries:
        country_id = region_id
        country_ids.append(country_id)
        regions_dict[country_id] = {
            'region_id': country_id,
            'parent_id': 0,
            'region_name': country,
            'region_type': 'country',
            'region_code': fake.country_code(),
            'population': random.randint(10000000, 150000000),
            'area_km2': round(random.uniform(100000, 2000000), 2),
            'updated_at': now
        }
        regions.append(regions_dict[country_id])
        region_id += 1
    
    # Регионы (для России)
    russia_id = country_ids[0]
    region_names = ['Московская область', 'Ленинградская область', 'Свердловская область', 
                    'Новосибирская область', 'Краснодарский край', 'Республика Татарстан']
    region_ids = []
    
    for region_name in region_names:
        region_id_val = region_id
        region_ids.append(region_id_val)
        region_code = f'RU-{fake.postcode()[:3].upper()}'
        regions_dict[region_id_val] = {
            'region_id': region_id_val,
            'parent_id': russia_id,
            'region_name': region_name,
            'region_type': 'region',
            'region_code': region_code,
            'population': random.randint(1000000, 10000000),
            'area_km2': round(random.uniform(10000, 200000), 2),
            'updated_at': now
        }
        regions.append(regions_dict[region_id_val])
        region_id += 1
    
    # Города (для каждого региона)
    city_names = ['Москва', 'Санкт-Петербург', 'Екатеринбург', 'Новосибирск', 
                  'Краснодар', 'Казань', 'Подольск', 'Химки', 'Гатчина', 'Нижний Тагил']
    
    for idx, region_id_val in enumerate(region_ids[:len(city_names)]):
        city_name = city_names[idx]
        city_code = f'RU-{fake.postcode()[:3].upper()}'
        regions_dict[region_id] = {
            'region_id': region_id,
            'parent_id': region_id_val,
            'region_name': city_name,
            'region_type': 'city',
            'region_code': city_code,
            'population': random.randint(100000, 15000000),
            'area_km2': round(random.uniform(100, 5000), 2),
            'updated_at': now
        }
        regions.append(regions_dict[region_id])
        region_id += 1
    
    # Вставка данных - формируем список списков в правильном порядке
    data = []
    for r in regions:
        data.append([
            r['region_id'],
            r['parent_id'],
            r['region_name'],
            r['region_type'],
            r['region_code'],
            r['population'],
            r['area_km2'],
            r['updated_at']
        ])
    
    client.insert('stage_regions_source', data, column_names=[
        'region_id', 'parent_id', 'region_name', 'region_type', 
        'region_code', 'population', 'area_km2', 'updated_at'
    ])
    
    print(f"✓ Сгенерировано {len(regions)} регионов")
    return region_ids


def generate_warehouses(client, region_ids):
    """Генерация складов"""
    print("\n🏭 Генерация складов...")
    
    warehouses = []
    warehouse_names = ['Склад Северный', 'Склад Южный', 'Склад Восточный', 'Склад Западный',
                       'Центральный склад', 'Склад Логистика-1', 'Склад Логистика-2',
                       'Склад Экспресс', 'Склад Оптима', 'Склад Премиум']
    
    now = datetime.now()
    
    for i, name in enumerate(warehouse_names):
        warehouse_id = f'WH{i+1:03d}'
        warehouses_list.append(warehouse_id)
        warehouses.append([
            warehouse_id,
            name,
            random.choice(region_ids),
            fake.address(),
            round(random.uniform(1000, 50000), 2),  # capacity_m3
            1,  # is_active
            now,
            now
        ])
    
    client.insert('stage_warehouses_source', warehouses, column_names=[
        'warehouse_id', 'warehouse_name', 'region_id', 'address',
        'capacity_m3', 'is_active', 'created_at', 'updated_at'
    ])
    
    print(f"✓ Сгенерировано {len(warehouses)} складов")
    return warehouses_list


def generate_suppliers(client):
    """Генерация поставщиков"""
    print("\n🏢 Генерация поставщиков...")
    
    suppliers = []
    supplier_names = ['Поставщик Альфа', 'Поставщик Бета', 'Поставщик Гамма', 'Поставщик Дельта',
                      'Торговый дом Вектор', 'ООО Логистик', 'ИП Иванов', 'ООО Снабжение',
                      'ТК Транзит', 'ООО Глобал', 'ИП Петров', 'ООО Мега',
                      'ТД Оптима', 'ООО Премиум', 'ИП Сидоров']
    
    now = datetime.now()
    
    for i, name in enumerate(supplier_names):
        supplier_id = f'SUP{i+1:03d}'
        suppliers_list.append(supplier_id)
        suppliers.append([
            supplier_id,
            name,
            fake.country(),
            round(random.uniform(3.0, 10.0), 2),  # rating
            fake.email(),
            now,
            now
        ])
    
    client.insert('stage_suppliers_source', suppliers, column_names=[
        'supplier_id', 'supplier_name', 'country', 'rating', 'contact_email',
        'created_at', 'updated_at'
    ])
    
    print(f"✓ Сгенерировано {len(suppliers)} поставщиков")
    return suppliers_list


def generate_products(client):
    """Генерация товаров"""
    print("\n📦 Генерация товаров...")
    
    products = []
    categories = ['electronics', 'clothing', 'food', 'books', 'toys', 'sports', 'home', 'beauty']
    
    product_names = [
        'Смартфон', 'Ноутбук', 'Планшет', 'Наушники', 'Клавиатура',
        'Футболка', 'Джинсы', 'Кроссовки', 'Куртка', 'Шапка',
        'Хлеб', 'Молоко', 'Яйца', 'Мясо', 'Рыба',
        'Книга', 'Журнал', 'Тетрадь', 'Ручка', 'Карандаш',
        'Мяч', 'Кукла', 'Конструктор', 'Пазл', 'Машинка',
        'Гантели', 'Мяч футбольный', 'Ракетка', 'Велосипед', 'Скейтборд',
        'Стол', 'Стул', 'Диван', 'Кровать', 'Шкаф',
        'Крем', 'Шампунь', 'Мыло', 'Зубная паста', 'Дезодорант'
    ]
    
    now = datetime.now()
    
    for i, name in enumerate(product_names):
        product_id = f'PROD{i+1:03d}'
        products_list.append(product_id)
        category = categories[i % len(categories)]
        products.append([
            product_id,
            name,
            category,
            round(random.uniform(100, 50000), 2),  # unit_price
            round(random.uniform(0.1, 50), 2),  # weight_kg
            round(random.uniform(0.001, 1), 3),  # volume_m3
            now,
            now
        ])
    
    # Дополняем до нужного количества
    while len(products) < NUM_PRODUCTS:
        product_id = f'PROD{len(products)+1:03d}'
        products_list.append(product_id)
        products.append([
            product_id,
            fake.word().capitalize() + ' ' + fake.word(),
            random.choice(categories),
            round(random.uniform(100, 50000), 2),
            round(random.uniform(0.1, 50), 2),
            round(random.uniform(0.001, 1), 3),
            now,
            now
        ])
    
    client.insert('stage_products_source', products, column_names=[
        'product_id', 'product_name', 'category', 'unit_price',
        'weight_kg', 'volume_m3', 'created_at', 'updated_at'
    ])
    
    print(f"✓ Сгенерировано {len(products)} товаров")
    return products_list


def generate_customers():
    """Генерация списка клиентов"""
    print("\n👥 Генерация клиентов...")
    
    for i in range(NUM_CUSTOMERS):
        customers_list.append(f'CUST{i+1:05d}')
    
    print(f"✓ Сгенерировано {len(customers_list)} клиентов")
    return customers_list


def generate_customer_segments(client):
    """Генерация сегментов клиентов"""
    print("\n📊 Генерация сегментов клиентов...")
    
    segments = []
    segment_types = ['vip', 'regular', 'new', 'inactive']
    
    # Выбираем случайных клиентов для сегментации
    selected_customers = random.sample(customers_list, NUM_CUSTOMER_SEGMENTS)
    now = datetime.now()
    
    for customer_id in selected_customers:
        segment = random.choice(segment_types)
        segments.append([
            customer_id,
            segment,
            round(random.uniform(0, 100), 2),  # segment_score
            fake.date_between(start_date='-1y', end_date='today'),  # assigned_date
            now
        ])
    
    client.insert('stage_customer_segments', segments, column_names=[
        'customer_id', 'segment', 'segment_score', 'assigned_date', 'updated_at'
    ])
    
    print(f"✓ Сгенерировано {len(segments)} сегментов клиентов")
    return segments


def generate_product_prices(client):
    """Генерация цен товаров"""
    print("\n💰 Генерация цен товаров...")
    
    prices = []
    start_date = date.today() - timedelta(days=365)
    now = datetime.now()
    
    for _ in range(NUM_PRODUCT_PRICES):
        product_id = random.choice(products_list)
        supplier_id = random.choice(suppliers_list)
        price_date = fake.date_between(start_date=start_date, end_date='today')
        
        # Получаем базовую цену из справочника товаров
        base_price = random.uniform(100, 50000)
        
        prices.append([
            product_id,
            price_date,
            round(base_price * random.uniform(0.8, 1.2), 2),  # price с вариацией
            supplier_id,
            'RUB',
            now
        ])
    
    client.insert('stage_product_prices', prices, column_names=[
        'product_id', 'price_date', 'price', 'supplier_id', 'currency', 'created_at'
    ])
    
    print(f"✓ Сгенерировано {len(prices)} записей цен")
    return prices


def generate_orders(client):
    """Генерация заказов"""
    print("\n🛒 Генерация заказов...")
    
    raw_orders = []
    stage_orders = []
    
    start_date = date.today() - timedelta(days=730)  # 2 года назад
    statuses = ['created', 'paid', 'shipped', 'delivered', 'cancelled']
    payment_methods = ['card', 'cash', 'transfer']
    tags_pool = ['urgent', 'vip', 'express', 'standard', 'bulk', 'fragile']
    now = datetime.now()
    
    for i in range(NUM_ORDERS):
        order_id = f'ORD{i+1:06d}'
        customer_id = random.choice(customers_list)
        order_date = fake.date_between(start_date=start_date, end_date='today')
        # Исправлено: используем date_time_between вместо combine
        order_datetime = fake.date_time_between(
            start_date=datetime.combine(order_date, datetime.min.time()),
            end_date=datetime.combine(order_date, datetime.max.time())
        )
        
        # Генерируем товары в заказе
        num_items = random.randint(1, 5)
        selected_products = random.sample(products_list, num_items)
        quantities = [random.randint(1, 10) for _ in range(num_items)]
        prices = [round(random.uniform(100, 5000), 2) for _ in range(num_items)]
        total_amount = sum(p * q for p, q in zip(prices, quantities))
        
        # Статус и история
        status = random.choice(statuses)
        status_history = ['created']
        if status in ['paid', 'shipped', 'delivered']:
            status_history.append('paid')
        if status in ['shipped', 'delivered']:
            status_history.append('shipped')
        if status == 'delivered':
            status_history.append('delivered')
        
        status_timestamps = [
            order_datetime + timedelta(hours=i) 
            for i in range(len(status_history))
        ]
        
        # Теги
        num_tags = random.randint(0, 3)
        tags = random.sample(tags_pool, num_tags) if num_tags > 0 else []
        
        # JSON метаданные
        metadata = {
            'payment_method': random.choice(payment_methods),
            'discount_percent': random.randint(0, 20),
            'special_instructions': fake.sentence() if random.random() > 0.7 else None
        }
        
        items_json = [
            {'product_id': p, 'quantity': q, 'price': pr}
            for p, q, pr in zip(selected_products, quantities, prices)
        ]
        
        # RAW заказ - ВАЖНО: порядок колонок должен соответствовать DDL
        raw_orders.append([
            order_id,                    # order_id String
            customer_id,                 # customer_id String
            order_date,                  # order_date Date
            order_datetime,              # order_datetime DateTime
            round(total_amount, 2),     # total_amount Decimal(10, 2)
            json.dumps(metadata, ensure_ascii=False),  # metadata_json String
            json.dumps(items_json, ensure_ascii=False),  # items_json String
            tags,                        # tags Array(String)
            selected_products,           # product_ids Array(String)
            quantities,                  # quantities Array(UInt32)
            prices,                      # prices Array(Decimal(10, 2))
            [random.randint(0, 15) for _ in range(num_items)],  # discount_percentages Array(UInt8)
            [order_date + timedelta(days=random.randint(1, 7))],  # delivery_dates Array(Date)
            status_history,              # status_history Array(String)
            status_timestamps,           # status_timestamps Array(DateTime)
            now,                         # created_at DateTime
            'api'                        # source String
        ])
        
        # STAGE заказ - ВАЖНО: порядок колонок должен соответствовать DDL
        stage_orders.append([
            order_id,                    # order_id String
            customer_id,                 # customer_id String
            order_date,                  # order_date Date
            order_datetime,              # order_datetime DateTime
            round(total_amount, 2),     # total_amount Decimal(10, 2)
            metadata['payment_method'],   # payment_method String
            status,                      # status String
            tags,                        # tags Array(String)
            selected_products,           # product_ids Array(String)
            quantities,                  # quantities Array(UInt32)
            prices,                      # prices Array(Decimal(10, 2))
            status_history,              # status_history Array(String)
            status_timestamps,           # status_timestamps Array(DateTime)
            now,                         # updated_at DateTime
            1                            # version UInt64
        ])
    
    # Вставка RAW заказов
    client.insert('raw_orders', raw_orders, column_names=[
        'order_id', 'customer_id', 'order_date', 'order_datetime', 'total_amount',
        'metadata_json', 'items_json', 'tags', 'product_ids', 'quantities', 'prices',
        'discount_percentages', 'delivery_dates', 'status_history', 'status_timestamps',
        'created_at', 'source'
    ])
    
    # Вставка STAGE заказов
    client.insert('stage_orders', stage_orders, column_names=[
        'order_id', 'customer_id', 'order_date', 'order_datetime', 'total_amount',
        'payment_method', 'status', 'tags', 'product_ids', 'quantities', 'prices',
        'status_history', 'status_timestamps', 'updated_at', 'version'
    ])
    
    print(f"✓ Сгенерировано {len(raw_orders)} заказов")
    return [o[0] for o in raw_orders]  # Возвращаем список order_id


def generate_shipments(client, order_ids):
    """Генерация поставок"""
    print("\n🚚 Генерация поставок...")
    
    # Проверка, что необходимые списки заполнены
    if not warehouses_list:
        raise ValueError("Список складов пуст. Сначала выполните generate_warehouses()")
    
    raw_shipments = []
    stage_shipments = []
    
    start_date = date.today() - timedelta(days=730)
    statuses = ['pending', 'in_transit', 'delivered', 'returned']
    checkpoints_pool = ['warehouse', 'transit', 'delivery', 'customs']
    now = datetime.now()
    
    # Связываем поставки с заказами
    shipment_orders = random.sample(order_ids, NUM_SHIPMENTS)
    
    for i, order_id in enumerate(shipment_orders):
        shipment_id = f'SHIP{i+1:06d}'
        warehouse_id = random.choice(warehouses_list)
        shipment_date = fake.date_between(start_date=start_date, end_date='today')
        # Исправлено: используем date_time_between вместо combine
        shipment_datetime = fake.date_time_between(
            start_date=datetime.combine(shipment_date, datetime.min.time()),
            end_date=datetime.combine(shipment_date, datetime.max.time())
        )
        
        # Упаковки
        num_packages = random.randint(1, 5)
        package_ids = [f'PKG{i+1:06d}-{j+1}' for j in range(num_packages)]
        package_weights = [round(random.uniform(0.5, 50), 2) for _ in range(num_packages)]
        total_weight = sum(package_weights)
        
        # Трекинг номера
        tracking_numbers = [f'TRK{random.randint(100000, 999999)}' for _ in range(num_packages)]
        
        # Точки контроля
        num_checkpoints = random.randint(2, 4)
        checkpoints = random.sample(checkpoints_pool, num_checkpoints)
        checkpoint_times = [
            shipment_datetime + timedelta(hours=random.randint(1, 48))
            for _ in range(num_checkpoints)
        ]
        
        # Транспорт
        vehicle_ids = [f'VEH{random.randint(100, 999)}'] if random.random() > 0.3 else []
        driver_ids = [f'DRV{random.randint(100, 999)}'] if random.random() > 0.3 else []
        
        status = random.choice(statuses)
        
        # JSON данные
        route_json = {
            'waypoints': [fake.city() for _ in range(random.randint(2, 5))],
            'distance_km': random.randint(50, 2000),
            'estimated_duration_hours': random.randint(2, 48)
        }
        
        tracking_json = {
            'events': [
                {'timestamp': str(dt), 'location': fake.city(), 'status': cp}
                for dt, cp in zip(checkpoint_times, checkpoints)
            ]
        }
        
        # RAW поставка - ВАЖНО: порядок колонок должен соответствовать DDL
        raw_shipments.append([
            shipment_id,                 # shipment_id String
            order_id,                    # order_id String
            warehouse_id,                # warehouse_id String
            shipment_date,               # shipment_date Date
            shipment_datetime,            # shipment_datetime DateTime
            json.dumps(route_json, ensure_ascii=False),  # route_json String
            json.dumps(tracking_json, ensure_ascii=False),  # tracking_json String
            package_ids,                 # package_ids Array(String)
            package_weights,             # package_weights Array(Float32)
            tracking_numbers,            # tracking_numbers Array(String)
            checkpoints,                 # checkpoints Array(String)
            checkpoint_times,            # checkpoint_times Array(DateTime)
            vehicle_ids,                # vehicle_ids Array(String)
            driver_ids,                 # driver_ids Array(String)
            now,                        # created_at DateTime
            'api'                       # source String
        ])
        
        # STAGE поставка - ВАЖНО: порядок колонок должен соответствовать DDL
        stage_shipments.append([
            shipment_id,                 # shipment_id String
            order_id,                    # order_id String
            warehouse_id,                # warehouse_id String
            shipment_datetime,            # shipment_datetime DateTime
            status,                      # status String
            round(total_weight, 2),     # weight Decimal(10, 2)
            package_ids,                 # package_ids Array(String)
            package_weights,             # package_weights Array(Float32)
            tracking_numbers,            # tracking_numbers Array(String)
            checkpoints,                 # checkpoints Array(String)
            checkpoint_times,            # checkpoint_times Array(DateTime)
            1,                          # sign Int8
            now                          # updated_at DateTime
        ])
    
    # Вставка RAW поставок
    client.insert('raw_shipments', raw_shipments, column_names=[
        'shipment_id', 'order_id', 'warehouse_id', 'shipment_date', 'shipment_datetime',
        'route_json', 'tracking_json', 'package_ids', 'package_weights', 'tracking_numbers',
        'checkpoints', 'checkpoint_times', 'vehicle_ids', 'driver_ids', 'created_at', 'source'
    ])
    
    # Вставка STAGE поставок
    client.insert('stage_shipments', stage_shipments, column_names=[
        'shipment_id', 'order_id', 'warehouse_id', 'shipment_datetime',
        'status', 'weight', 'package_ids', 'package_weights', 'tracking_numbers',
        'checkpoints', 'checkpoint_times', 'sign', 'updated_at'
    ])
    
    print(f"✓ Сгенерировано {len(raw_shipments)} поставок")
    return [s[0] for s in raw_shipments]  # Возвращаем список shipment_id


def generate_inventory_snapshots(client):
    """Генерация снимков инвентаря"""
    print("\n📦 Генерация снимков инвентаря...")
    
    snapshots = []
    start_date = datetime.now() - timedelta(days=90)  # Последние 90 дней
    now = datetime.now()
    
    # Генерируем снимки для разных комбинаций warehouse + product
    combinations = []
    for warehouse_id in warehouses_list:
        for product_id in random.sample(products_list, random.randint(10, 30)):
            combinations.append((warehouse_id, product_id))
    
    # Генерируем несколько снимков для каждой комбинации
    for warehouse_id, product_id in combinations[:NUM_INVENTORY_SNAPSHOTS // 3]:
        num_snapshots = random.randint(1, 5)
        base_quantity = random.randint(0, 1000)
        
        for i in range(num_snapshots):
            snapshot_time = start_date + timedelta(
                days=random.randint(0, 90),
                hours=random.randint(0, 23)
            )
            quantity = max(0, base_quantity + random.randint(-100, 100))
            
            # Метаданные
            metadata = {
                'batch_info': f'BATCH-{random.randint(1000, 9999)}',
                'quality_control': random.choice(['passed', 'pending', 'failed'])
            }
            
            # Локации
            num_locations = random.randint(1, 3)
            location_codes = [f'{chr(65+i)}-{random.randint(1,10)}-{random.randint(1,20)}' 
                             for i in range(num_locations)]
            
            # Партии
            num_batches = random.randint(1, 2)
            batch_numbers = [f'BATCH-{random.randint(1000, 9999)}' for _ in range(num_batches)]
            expiry_dates = [
                date.today() + timedelta(days=random.randint(1, 365))
                for _ in range(num_batches)
            ]
            supplier_ids = random.sample(suppliers_list, num_batches)
            cost_prices = [round(random.uniform(50, 500), 2) for _ in range(num_batches)]
            
            # RAW инвентарь - ВАЖНО: порядок колонок должен соответствовать DDL
            snapshots.append([
                f'INV{len(snapshots)+1:08d}',  # inventory_id String
                warehouse_id,                   # warehouse_id String
                product_id,                     # product_id String
                snapshot_time,                  # snapshot_time DateTime
                quantity,                       # quantity UInt32
                json.dumps(metadata, ensure_ascii=False),  # metadata_json String
                location_codes,                 # location_codes Array(String)
                batch_numbers,                  # batch_numbers Array(String)
                expiry_dates,                   # expiry_dates Array(Date)
                supplier_ids,                    # supplier_ids Array(String)
                cost_prices,                    # cost_prices Array(Decimal(10, 2))
                now,                            # created_at DateTime
                'api'                           # source String
            ])
    
    # Дополняем до нужного количества
    while len(snapshots) < NUM_INVENTORY_SNAPSHOTS:
        warehouse_id = random.choice(warehouses_list)
        product_id = random.choice(products_list)
        snapshot_time = start_date + timedelta(
            days=random.randint(0, 90),
            hours=random.randint(0, 23)
        )
        quantity = random.randint(0, 1000)
        
        metadata = {'batch_info': f'BATCH-{random.randint(1000, 9999)}'}
        location_codes = [f'{chr(65+random.randint(0,5))}-{random.randint(1,10)}-{random.randint(1,20)}']
        batch_numbers = [f'BATCH-{random.randint(1000, 9999)}']
        expiry_dates = [date.today() + timedelta(days=random.randint(1, 365))]
        supplier_ids = [random.choice(suppliers_list)]
        cost_prices = [round(random.uniform(50, 500), 2)]
        
        snapshots.append([
            f'INV{len(snapshots)+1:08d}',
            warehouse_id,
            product_id,
            snapshot_time,
            quantity,
            json.dumps(metadata, ensure_ascii=False),
            location_codes,
            batch_numbers,
            expiry_dates,
            supplier_ids,
            cost_prices,
            now,
            'api'
        ])
    
    # Вставка RAW инвентаря
    client.insert('raw_inventory', snapshots, column_names=[
        'inventory_id', 'warehouse_id', 'product_id', 'snapshot_time', 'quantity',
        'metadata_json', 'location_codes', 'batch_numbers', 'expiry_dates',
        'supplier_ids', 'cost_prices', 'created_at', 'source'
    ])
    
    # Генерируем STAGE снимки (выборочно для ASOF JOIN)
    stage_snapshots = []
    for snapshot in random.sample(snapshots, min(5000, len(snapshots) // 2)):
        stage_snapshots.append([
            snapshot[1],  # warehouse_id
            snapshot[2],  # product_id
            snapshot[3],  # snapshot_time
            snapshot[4],  # quantity
            random.randint(0, snapshot[4] // 2),  # reserved_quantity
            snapshot[4] - random.randint(0, snapshot[4] // 2),  # available_quantity
            now
        ])
    
    client.insert('stage_inventory_snapshots', stage_snapshots, column_names=[
        'warehouse_id', 'product_id', 'snapshot_time', 'quantity',
        'reserved_quantity', 'available_quantity', 'created_at'
    ])
    
    print(f"✓ Сгенерировано {len(snapshots)} снимков инвентаря (RAW: {len(snapshots)}, STAGE: {len(stage_snapshots)})")
    return snapshots


def generate_transport(client, shipment_ids):
    """Генерация транспорта"""
    print("\n🚛 Генерация транспорта...")
    
    transport_records = []
    now = datetime.now()
    
    # Связываем транспорт с поставками
    selected_shipments = random.sample(shipment_ids, min(len(shipment_ids), 3000))
    
    for shipment_id in selected_shipments:
        departure_time = fake.date_time_between(start_date='-1y', end_date='now')
        arrival_time = departure_time + timedelta(
            hours=random.randint(2, 72),
            minutes=random.randint(0, 59)
        )
        
        # JSON данные - ВАЖНО: преобразуем все числа в float для JSON сериализации
        route_json = {
            'coordinates': [
                {'lat': float(fake.latitude()), 'lon': float(fake.longitude())}
                for _ in range(random.randint(2, 5))
            ],
            'stops': [fake.city() for _ in range(random.randint(1, 3))],
            'traffic_delay_minutes': random.randint(0, 120)
        }
        
        vehicle_info_json = {
            'type': random.choice(['truck', 'van', 'container']),
            'capacity_kg': random.randint(1000, 20000),
            'driver': {
                'name': fake.name(),
                'license': f'DL{random.randint(100000, 999999)}'
            }
        }
        
        # RAW транспорт - ВАЖНО: порядок колонок должен соответствовать DDL
        transport_records.append([
            f'TRANS{len(transport_records)+1:06d}',  # transport_id String
            shipment_id,                              # shipment_id String
            departure_time,                          # departure_time DateTime
            arrival_time,                            # arrival_time DateTime
            json.dumps(route_json, ensure_ascii=False),  # route_json String
            json.dumps(vehicle_info_json, ensure_ascii=False),  # vehicle_info_json String
            now,                                     # created_at DateTime
            'api'                                    # source String
        ])
    
    client.insert('raw_transport', transport_records, column_names=[
        'transport_id', 'shipment_id', 'departure_time', 'arrival_time',
        'route_json', 'vehicle_info_json', 'created_at', 'source'
    ])
    
    print(f"✓ Сгенерировано {len(transport_records)} записей транспорта")
    return transport_records


def generate_mart_tables(client):
    """Генерация данных для MART таблиц (исторические данные)"""
    print("\n📊 Генерация данных для MART таблиц...")
    
    # Получаем заказы из STAGE
    orders_2024 = []
    orders_2023 = []
    orders_2022 = []
    now = datetime.now()
    
    result = client.query("""
        SELECT order_id, customer_id, order_date, total_amount, status
        FROM stage_orders
        WHERE order_date >= '2022-01-01'
        ORDER BY order_date
    """)
    
    for row in result.result_rows:
        order_id, customer_id, order_date, total_amount, status = row
        year = order_date.year
        
        order_data = [
            order_id,
            customer_id,
            order_date,
            total_amount,
            status,
            now
        ]
        
        if year == 2024:
            orders_2024.append(order_data)
        elif year == 2023:
            orders_2023.append(order_data)
        elif year == 2022:
            orders_2022.append(order_data)
    
    # Вставка в соответствующие таблицы
    if orders_2024:
        client.insert('mart_orders_2024', orders_2024, column_names=[
            'order_id', 'customer_id', 'order_date', 'total_amount', 'status', 'created_at'
        ])
        print(f"✓ Вставлено {len(orders_2024)} заказов в mart_orders_2024")
    
    if orders_2023:
        client.insert('mart_orders_2023', orders_2023, column_names=[
            'order_id', 'customer_id', 'order_date', 'total_amount', 'status', 'created_at'
        ])
        print(f"✓ Вставлено {len(orders_2023)} заказов в mart_orders_2023")
    
    if orders_2022:
        client.insert('mart_orders_2022', orders_2022, column_names=[
            'order_id', 'customer_id', 'order_date', 'total_amount', 'status', 'created_at'
        ])
        print(f"✓ Вставлено {len(orders_2022)} заказов в mart_orders_2022")

In [60]:
def main():
    """Основная функция генерации данных"""
    print("=" * 60)
    print("ГЕНЕРАТОР ДАННЫХ ДЛЯ LOGISTICS & SUPPLY CHAIN")
    print("=" * 60)
    
    client = init_client()
    
    try:
        # Генерация справочников
        region_ids = generate_regions(client)
        warehouses_list.extend(generate_warehouses(client, region_ids))
        suppliers_list.extend(generate_suppliers(client))
        products_list.extend(generate_products(client))
        customers_list.extend(generate_customers())
        
        # Генерация зависимых данных
        generate_customer_segments(client)
        generate_product_prices(client)
        
        # Генерация транзакционных данных
        order_ids = generate_orders(client)
        shipment_ids = generate_shipments(client, order_ids)
        generate_inventory_snapshots(client)
        generate_transport(client, shipment_ids)
        
        # Генерация MART таблиц
        generate_mart_tables(client)
        
        print("\n" + "=" * 60)
        print("✓ ГЕНЕРАЦИЯ ДАННЫХ ЗАВЕРШЕНА УСПЕШНО!")
        print("=" * 60)
        
        # Статистика
        print("\n📈 Статистика:")
        stats = client.query("""
            SELECT 
                'raw_orders' as table_name, count() as rows FROM raw_orders
            UNION ALL
            SELECT 'raw_shipments', count() FROM raw_shipments
            UNION ALL
            SELECT 'raw_inventory', count() FROM raw_inventory
            UNION ALL
            SELECT 'raw_transport', count() FROM raw_transport
            UNION ALL
            SELECT 'stage_orders', count() FROM stage_orders
            UNION ALL
            SELECT 'stage_shipments', count() FROM stage_shipments
            UNION ALL
            SELECT 'stage_inventory_snapshots', count() FROM stage_inventory_snapshots
            UNION ALL
            SELECT 'stage_product_prices', count() FROM stage_product_prices
            UNION ALL
            SELECT 'stage_customer_segments', count() FROM stage_customer_segments
            UNION ALL
            SELECT 'stage_regions_source', count() FROM stage_regions_source
            UNION ALL
            SELECT 'stage_warehouses_source', count() FROM stage_warehouses_source
            UNION ALL
            SELECT 'stage_suppliers_source', count() FROM stage_suppliers_source
            UNION ALL
            SELECT 'stage_products_source', count() FROM stage_products_source
        """)
        
        for row in stats.result_rows:
            print(f"  {row[0]}: {row[1]:,} строк")
        
    except Exception as e:
        print(f"\n✗ Ошибка при генерации данных: {e}")
        import traceback
        traceback.print_exc()
    finally:
        client.close()


if __name__ == '__main__':
    main()

ГЕНЕРАТОР ДАННЫХ ДЛЯ LOGISTICS & SUPPLY CHAIN
✓ Подключение к ClickHouse установлено: localhost:8123

📍 Генерация регионов...
✓ Сгенерировано 15 регионов

🏭 Генерация складов...
✓ Сгенерировано 10 складов

🏢 Генерация поставщиков...
✓ Сгенерировано 15 поставщиков

📦 Генерация товаров...
✓ Сгенерировано 50 товаров

👥 Генерация клиентов...
✓ Сгенерировано 1400 клиентов

📊 Генерация сегментов клиентов...
✓ Сгенерировано 150 сегментов клиентов

💰 Генерация цен товаров...
✓ Сгенерировано 200 записей цен

🛒 Генерация заказов...
✓ Сгенерировано 5000 заказов

🚚 Генерация поставок...
✓ Сгенерировано 4000 поставок

📦 Генерация снимков инвентаря...
✓ Сгенерировано 10000 снимков инвентаря (RAW: 10000, STAGE: 5000)

🚛 Генерация транспорта...
✓ Сгенерировано 3000 записей транспорта

📊 Генерация данных для MART таблиц...
✓ Вставлено 2461 заказов в mart_orders_2024
✓ Вставлено 203 заказов в mart_orders_2023

✓ ГЕНЕРАЦИЯ ДАННЫХ ЗАВЕРШЕНА УСПЕШНО!

📈 Статистика:
  stage_product_prices: 200 строк
  raw_s